In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import shapefile as sf
from pyproj import Proj
from censusgeocode import CensusGeocode

import datetime

%matplotlib inline

In [11]:
code_violations = pd.read_csv('code_violations_records/mappedcedcases6months_complaints_datasd.csv')

In [12]:
code_violations.columns.values

array(['APN', 'case_id', 'case_source', 'close_date', 'close_note',
       'close_reason', 'description', 'complaint_type',
       'complaint_type_id', 'investigator_active',
       'investigator_email_address', 'investigator_name',
       'investigator_phone_number', 'last_action', 'last_action_due_date',
       'latitude', 'longitude', 'open_date', 'remedy_msg',
       'street_address', 'workgroup'], dtype=object)

In [13]:
len(code_violations)

5983

In [16]:
cg = CensusGeocode()

In [18]:
code_violations['block'] = None
code_violations['tract'] = None
code_violations['geoid'] = None
code_violations['blockgroup'] = None

In [22]:
start = datetime.datetime.now()

print len(code_violations)

for i in range(0,len(code_violations)): 
    if ((code_violations.loc[i,'longitude']!=0.0) 
        & (np.isnan(code_violations.loc[i,'longitude'])==False)) :
        temp1 = cg.coordinates(x=code_violations.loc[i,'longitude'], 
                              y=code_violations.loc[i, 'latitude'])
        if (len(temp1[0]['2010 Census Blocks'])>0):
            if ('status' not in temp1[0]['2010 Census Blocks'][0].keys()[0]): 
                temp = temp1[0]['2010 Census Blocks'][0]
                if (temp is not None):
                    code_violations.loc[i, 'block'] = temp['BLOCK']
                    code_violations.loc[i, 'tract'] = temp['TRACT']
                    code_violations.loc[i, 'geoid'] = temp['GEOID']
                    code_violations.loc[i, 'blockgroup'] = temp['BLKGRP']

                if (i % 100 == 0): 
                    finish = datetime.datetime.now()
                    print i, finish-start
                    start = datetime.datetime.now()



5983
0 0:00:01.907576
100 0:01:54.528919
200 0:02:18.308541
300 0:02:24.957082
400 0:02:58.417412
500 0:02:31.206979
600 0:02:03.234116
700 0:02:56.658819
800 0:02:50.491163
900 0:02:32.843516
1000 0:02:33.097257
1100 0:02:27.172335
1200 0:02:43.832626
1300 0:02:35.365162
1400 0:02:03.697580
1500 0:02:36.586664
1600 0:02:44.807966
1700 0:02:27.769980
1800 0:02:51.221879
1900 0:02:22.618915
2000 0:02:25.157750
2100 0:02:11.579909
2200 0:02:46.641649
2300 0:02:30.722565
2400 0:02:26.420384
2500 0:02:38.462333
2600 0:01:47.907124
2700 0:01:42.165460
2800 0:01:54.909988
2900 0:01:58.391362
3000 0:02:29.531169
3100 0:02:43.906707
3200 0:02:45.178666
3300 0:03:35.318252
3400 0:02:53.914088
3500 0:02:33.283590
3600 0:02:31.714271
3700 0:02:29.538437
3800 0:03:00.475451
3900 0:02:37.114294
4000 0:02:37.317314
4100 0:02:44.842207
4200 0:02:20.828974
4300 0:02:26.335431
4400 0:02:22.449399
4500 0:01:52.918738
4600 0:01:33.379216
4700 0:01:27.662122
4800 0:01:21.873635
4900 0:01:46.055481
5000 0:

In [41]:
code_violations.columns.values

array(['APN', 'case_id', 'case_source', 'close_date', 'close_note',
       'close_reason', 'description', 'complaint_type',
       'complaint_type_id', 'investigator_active',
       'investigator_email_address', 'investigator_name',
       'investigator_phone_number', 'last_action', 'last_action_due_date',
       'latitude', 'longitude', 'open_date', 'remedy_msg',
       'street_address', 'workgroup', 'block', 'tract', 'geoid',
       'blockgroup', 'cv_count'], dtype=object)

In [25]:
#code_violations.to_csv('output_files/code_violations_6mo_all.csv', index=False)

In [139]:
#code_violations = pd.read_csv('output_files/code_violations_all.csv')

Organise code violations by tract and block group

In [31]:
code_violations['cv_count'] = 1.0

In [28]:
code_violations['tract'] = code_violations['tract'].astype(float)
code_violations['blockgroup'] = code_violations['blockgroup'].astype(float)

In [39]:
cv_by_bg = code_violations[['tract', 'blockgroup', 'cv_count']].groupby(['tract','blockgroup'], as_index=False).agg('count')

In [40]:
#cv_by_bg.to_csv('output_files/code_violations_6mo_bg.csv', index=False)